In [24]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.forecasting.theta import ThetaModel
from scipy.optimize import fsolve
from scipy.special import rel_entr, kl_div
from sklearn.model_selection import ParameterGrid
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
# plt.style.use('fivethirtyeight')
# plt.style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_density_forecast(ts, horizon, base_alg, base_params={}, fit_params={}, exog=None,
                         bins='auto', omega=None, fittedvalues=False):
    """
    Returns a list of density dictionaries {'bins': np.array, 'probs': np.array, 'dotted_forecast': float}.
    
    Parameters
    ----------
    ts : array_like
        The time series to model.
    horizon : int
        The horizon to forecast.
    base_alg : {ExponentialSmoothing, SimpleExpSmoothing, Holt, ARIMA}
        The name of base algoritm for making density forecast.
    base_params : dict
        A Dictionary with base algorithm parameters.
    bins: int or sequence of scalars or str, optional
        Define how to calculate bins.
    fittedvalues: bool
        Include fitted values in density dictionaries or not.
    """
    
    if omega is not None:
        bins = omega
        
    if base_alg == ARIMA:
        alg = base_alg(ts, exog, **base_params).fit(**fit_params)
    else:
        alg = base_alg(ts, **base_params).fit(**fit_params)
    
    if fittedvalues:
        alg_preds = alg.predict(start=0, end=len(ts) + horizon - 1)
        density_dicts = [{'bins': [], 'probs': [], 'dotted_forecast': None} for _ in range(len(ts) + horizon)]
    else:
        alg_preds = alg.predict(start=len(ts), end=len(ts) + horizon - 1)
        density_dicts = [{'bins': [], 'probs': [], 'dotted_forecast': None} for _ in range(horizon)]
    
    for i in range(len(alg_preds)):
        density_dicts[i]['dotted_forecast'] = alg_preds.iloc[i]
        
        current_density = alg.resid + alg_preds.iloc[i]
        probs, bins = np.histogram(current_density, bins=bins)
        density_dicts[i]['probs'], density_dicts[i]['bins'] = probs / np.sum(probs), bins
    
    return density_dicts

In [3]:
def plot_density_forecast(ts, delay, base_alg, base_params={}, fit_params={},
                          ax=None, **kwargs):
        
    density_dict = get_density_forecast(ts, delay, base_alg, base_params, fit_params, **kwargs)[delay - 1]

    left_edges = density_dict['bins'][:-1]

    colors = []
    for i in range(len(left_edges) - 1):
        if left_edges[i] < density_dict['dotted_forecast'] < left_edges[i+1]:
            colors.append('coral')
        else:
            colors.append('royalblue')
    
    alg_name = str(base_alg)[str(base_alg).find('model.') + 6:-2]
    
    if ax:
        ax.bar(left_edges, density_dict['probs'], align='edge',
               width=0.9*(left_edges[1] - left_edges[0]), color=colors)
        ax.set_title(f'{alg_name}: density forecast with delay={delay}')
    else:
        plt.bar(left_edges, density_dict['probs'], align='edge',
               width=0.9*(left_edges[1] - left_edges[0]), color=colors)
        plt.title(f'{alg_name}: density forecast with delay={delay}')

In [4]:
def get_omega(ts, mode, bins=None, quantile=0.1):
    if mode == "basic":
        return np.histogram_bin_edges(ts, bins)
    if mode == "quantile":
        bin_width = int(np.quantile(abs(ts - ts.shift(1))[1:], quantile))
    elif mode == "mean":
        bin_width = int(np.mean(abs(ts - ts.shift(1))[1:]))
    
    if bin_width < 1:
        bin_width = 1
    
    min_o = int(np.floor(min(ts.values)))
    max_o = int(np.ceil(max(ts.values)))
    
    bin_edges = [min_o]
    while bin_edges[-1] < max_o:
        bin_edges.append(bin_edges[-1] + bin_width)
    
    return np.array(bin_edges)

In [5]:
def brier_loss(y_true, density_dict, p=2):
    """
    Returns np.array of brier scores.
    
    Parameters
    ----------
    y_true : float
        A true value.
    density_dict : dict
        Dict with bins and probabilities information.
    """
    bins_number = density_dict['probs'].size
    bins_true = [0] * bins_number

    for i in range(bins_number):
        if (density_dict['bins'][i] <= y_true <= density_dict['bins'][i + 1]):
            bins_true[i] = 1
            break
    
    brier_loss = np.sum((abs(density_dict['probs'] - bins_true))**p)
    
    if np.sum(bins_true) == 0:
        brier_loss += 1
        
    return brier_loss

In [6]:
def get_generalized_loss(y_true, density_dicts, loss_function, p):
    """
    Returns np.array of brier scores.
    
    Parameters
    ----------
    y_true : float
        A true value.
    density_dicts : dict or array-like of dicts
        Dicts with bins and probabilities information.
    """
    if type(density_dicts) == dict:
        density_dicts = [density_dicts]
    
    losses = [np.nan] * len(density_dicts)
    
    for density_dict_count, density_dict in enumerate(density_dicts):
        losses[density_dict_count] = loss_function(y_true, density_dict, p)
    
    return np.array(losses)

In [7]:
def avoid_overflowing(base, power_array):
    maximum = np.max(power_array)
    minimum = np.min(power_array)
    
    pmax = -np.log(base)/np.log(2) * maximum
    pmin = -np.log(base)/np.log(2) * minimum
    
    if np.abs(pmax-pmin) > 2097:
        print('Overflow is imminent. Further calculations are not advised')
        return base ** power_array
    power_shift = abs((51+pmin+pmax)/2)
    power_shift = power_shift + min(0, pmin - power_shift + 1023)
    
    power_array = power_array - np.abs(power_shift * np.log(2) / np.log(base))
    
    return base ** power_array

In [8]:
def get_generalized_prediction(ts, preds, omega, weights, loss_function, p, eta):
    generalized_predictions = []
    
    for w in zip(omega, omega[1:]):
        losses = get_generalized_loss((w[0] + w[1]) / 2, preds, loss_function, p)
        exp_losses = avoid_overflowing(np.e, -eta * losses)
        generalized_predictions.append(-(1 / eta) * np.log(np.sum(weights * exp_losses)))
        
    return np.array(generalized_predictions)

In [9]:
from scipy.optimize import fsolve

def s_equation(s, generalized_predictions, m=2):
    return np.sum([max(x,0) for x in s - generalized_predictions]) - m

In [10]:
def substitution_function(generalized_predictions, s, m=2):
    predictions = [max(x,0) / m for x in (s - generalized_predictions)]
    return np.array(predictions)

In [11]:
def update_weights(weights, losses, eta=1):
    exp_losses = avoid_overflowing(np.e, -eta * np.array(losses))
    new_weights = weights * exp_losses
    return new_weights / (np.sum(new_weights))

In [12]:
def aggregating_algorithm(ts, horizon, base_alg_list, bins=10, omega_mode="basic",
                          loss_function=brier_loss, exog=None, m=2, p=2, weights=None, eta=1):
    """
    Returns density dictionary {'bins': np.array, 'probs': np.array, 'dotted_forecast': float}.
    
    Parameters
    ----------
    ts : array_like
        The time series to model.
    horizon : int
        The delay to forecast.
    base_alg_list : dict
        The dictionary with the names of base algoritms and their params:
        base_alg {ExponentialSmoothing, SimpleExpSmoothing, Holt} - name of base algorithm.
        base_alg_params : dict - a dictionary of base algorithm's parameters.
    loss_function : function
        The loss function of aggregating algorithm.
    """
    
    T = len(ts)
    K = len(base_alg_list)
    
    AA_preds = [{} for i in range(T + horizon)]
    BA_preds = np.array([{} for i in range((T + horizon) * K)]).reshape(K, T + horizon) 
    
    if not weights:
        weights = np.full(K, 1/K)
    
    omega = get_omega(ts, mode=omega_mode, bins=bins) # здесь в Omega прогнозы не учитываются
 
    i = 0
    for base_alg, base_alg_params, fit_alg_params in base_alg_list:
        BA_preds[i] = get_density_forecast(ts, horizon, base_alg, base_params=base_alg_params,
                                           fit_params=fit_alg_params, omega=omega, exog=exog, fittedvalues=True)
        i += 1
        
        
    losses, prev_losses = None, None
        
    for t in tqdm(range(T + horizon), leave=False):
        preds = BA_preds[:, t]
        
        if not prev_losses:
            prev_losses = [loss_function(ts.values[t], pred, p) for pred in preds]  # cheat
              
        generalized_predictions = get_generalized_prediction(ts.values[:t], preds, omega,
                                                             weights, loss_function, p, eta)
        
        #solving the equation to find s
        s_init = np.max(generalized_predictions)
        s = fsolve(s_equation, s_init, args=(generalized_predictions, m))
        
        #get real prediction with substitution function
        AA_preds[t]['bins'] = BA_preds[:, t][0]['bins']
        real_predictions = substitution_function(generalized_predictions, s, m)
        AA_preds[t]['probs'] = real_predictions
        
        #update weights 
        if t < T:            
            losses = [loss_function(ts.values[t], pred, p) for pred in preds]
            weights = update_weights(weights, losses, eta)

            prev_losses = losses
    
    return AA_preds

In [13]:
def get_optim_m(ts, base_alg_list, omega_mode="basic",
                exog=None, bins=10, p=2):
    best_m = 0
    best_loss = 1000
    AA_losses = {}
    # (np.linspace(0.1, 10, 100),
    for m in tqdm(np.linspace(0.5, 10, 20), leave=False):
        AA_preds = aggregating_algorithm(ts, 0, base_alg_list,
                                         bins=bins, omega_mode=omega_mode, exog=exog, m=m, p=p)
        AA_losses[m] = []
        for i in range(len(ts.values)):
            AA_losses[m].append((brier_loss(ts.values[i], AA_preds[i])))
        loss = np.mean(AA_losses[m])

        if loss < best_loss:
            best_m = m
            best_loss = loss
            
    n_bins = AA_preds[0]['probs'].size
    return best_m, best_loss, n_bins, AA_losses

In [14]:
def plot_optim_m(all_losses_dict, best_m, ax=plt):
    all_losses = [(key, np.mean(values)) for key, values in all_losses_dict.items()]
    m_list, loss_list = zip(*all_losses)

    ax.plot(m_list, loss_list)
    
    x, y = best_m, np.mean(all_losses_dict[best_m])
    ax.scatter(x, y, s=70, c="orange", zorder=3)
    ax.text(x - 0.4, y + 0.02, '({}, {})'.format(x, round(y, 2)));

In [77]:
def get_opt_arima(endog, exog, grid_params):
    opt_model = {'opt_params': {'order': (0, 0, 0), 'seasonal_order': (0, 0, 0, 0)},
                 'opt_bic': 10**6, 'opt_model': None}
    
    # 'opt_params': {'p': 0, 'd':0, 'q': 0, 'P': 0, 'D':0, 'Q': 0, 's': 0}

    grid = ParameterGrid(grid_params)
    for params in tqdm(grid, leave=False):
        try:
            arima = ARIMA(endog,
                      exog,
                      order=(params['p'], params['d'], params['q']),
                      seasonal_order=(params['P'], params['D'], params['Q'], params['s'])).fit()
        except:
            print("LU decomposition error")
            continue

        if arima.bic < opt_model['opt_bic']:
            opt_model['opt_params']['order'] = (params['p'], params['d'], params['q'])
            opt_model['opt_params']['seasonal_order'] = (params['P'], params['D'], params['Q'], params['s'])
            opt_model['opt_bic'] = arima.bic
            opt_model['opt_model'] = arima
    
    return opt_model

In [ ]:
def get_auto_base_alg_list(endog, exog, grid_params):
    auto_base_alg_list = []
    
    opt_arima = get_opt_arima(endog, exog, grid_params)
    auto_base_alg_list.append((ARIMA, opt_arima['opt_params'], {}))
    
    SES = ExponentialSmoothing(endog).fit(optimized=True)
    auto_base_alg_list.append((ExponentialSmoothing, {},
                               {'smoothing_level': SES.params['smoothing_level']}))
    
    ES_add7 = ExponentialSmoothing(endog, seasonal="add",
                                   seasonal_periods=7).fit(optimized=True)
    auto_base_alg_list.append((ExponentialSmoothing, {"seasonal": "add", "seasonal_periods": 7},
                               {'smoothing_level': ES_add7.params['smoothing_level'],
                                'smoothing_seasonal': ES_add7.params['smoothing_seasonal']}))
    
    return auto_base_alg_list

In [136]:
def plot_losses(ts, horizon, base_alg_list, omega_mode, best_m, ax, bins=10, exog=None,
                title='', legend=[], number_to_plot=None):
    omega = get_omega(ts, mode=omega_mode, bins=bins)
    
    pred_dict = {}
    i = 1
    for base_alg, base_params, fit_params in base_alg_list:
        pred_dict[f"alg{i}"] = get_density_forecast(ts, horizon, base_alg, base_params, fit_params,
                                                    bins=bins, omega=omega, exog=exog, fittedvalues=True)
        i += 1
    
    pred_dict["AA"] = aggregating_algorithm(ts, horizon, base_alg_list, exog=exog,
                                            bins=bins, omega_mode=omega_mode, m=best_m)

    losses_dict = {key : [] for key in pred_dict.keys()}
    
    for i in range(len(ts.values)):
        for alg in losses_dict.keys():
            losses_dict[alg].append(brier_loss(ts.values[i], pred_dict[alg][i]))
          
    for alg in losses_dict.keys():
        losses_dict[alg] = np.cumsum(losses_dict[alg]) / list(range(1, len(ts.values) + 1))
        
    theoretical_bound = (np.array(list(losses_dict.values())).min(axis=0)
                         + np.log(len(losses_dict)) / list(range(1, len(ts.values) + 1)))
    
    tb_errors = (losses_dict["AA"] > theoretical_bound).sum()
    if tb_errors == 0:
        print("Theoretical bound met")
    else:
        print(f"Theoretical bound not met: {tb_errors / ts.values.size}% violations")
        
    losses_dict_sorted = {k: v for k, v in sorted(losses_dict.items(), key=lambda item: item[1].mean())}
    
    if not number_to_plot:
        number_to_plot = len(base_alg_list)
    
    real_legend = []
    i = 0
    for alg in losses_dict_sorted.keys():
        if i < number_to_plot:
            if alg == "AA":
                continue
            ax.plot(ts.index, losses_dict_sorted[alg])
            i += 1
            real_legend.append(legend[int(alg[-1]) - 1])
            print(f"{legend[int(alg[-1]) - 1]} mean loss: {losses_dict_sorted[alg].mean()}")
        else:
            break
        
    ax.plot(ts.index, losses_dict_sorted['AA'])
    print(f"AA mean loss: {losses_dict_sorted['AA'].mean()}")
    real_legend.append('AA')
    ax.plot(ts.index, theoretical_bound)
    real_legend.append('TB')
    
    ax.set_title(title)
    ax.legend(real_legend, loc='upper right');
    return losses_dict_sorted

In [170]:
def get_all_predictions(train_dict, test_dict, horizon, base_alg_list, grid_params,
                        omega_mode="basic", best_m="optim", is_exog=False, bins=10):
    pred_dict = {}
    losses_dict = {}
    
    for ts_name in tqdm(train_dict.keys()):
        if is_exog:
            ts_train = train_dict[ts_name]["ts"]
            ts_test = test_dict[ts_name]["ts"]
            exog_train = (pd.DataFrame(data=[train_dict[ts_name]["actual_price"],
                                             train_dict[ts_name]["promo"]])
                          .transpose().astype({"Actual_Price": float, "Promo": int}))
            exog_test = (pd.DataFrame(data=[test_dict[ts_name]["actual_price"],
                                            test_dict[ts_name]["promo"]])
                         .transpose().astype({"Actual_Price": float, "Promo": int}))
        else:
            ts_train = train_dict[ts_name]
            ts_test = tesr_dict[ts_name]
            exog_train = None
            exog_test = None
        
        auto_base_alg_list = get_auto_base_alg_list(ts_train, exog_train, grid_params)
        all_base_alg_list = base_alg_list + auto_base_alg_list
        
        if best_m == "optim":
            best_m, _, _, _ = get_optim_m(ts_train, all_base_alg_list, exog=exog_train,
                                          bins=bins, omega_mode=omega_mode)
        
        omega = get_omega(ts_test, mode=omega_mode, bins=bins)
        
        pred_dict[ts_name] = {}
        i = 1
        for base_alg, base_params, fit_params in all_base_alg_list:
            pred_dict[ts_name][f"alg{i}"] = get_density_forecast(ts_test, horizon, base_alg, base_params, fit_params,
                                                                 exog=exog_test, bins=bins,
                                                                 omega=omega, fittedvalues=True)
            i += 1

        pred_dict[ts_name]["AA"] = aggregating_algorithm(ts_test, horizon, all_base_alg_list, exog=exog_test,
                                                         bins=bins, omega_mode=omega_mode, m=best_m)

        losses_dict[ts_name] = {key : [] for key in pred_dict[ts_name].keys()}

        for i in range(len(ts_test.values)):
            for alg in losses_dict[ts_name].keys():
                losses_dict[ts_name][alg].append(brier_loss(ts_test.values[i], pred_dict[ts_name][alg][i]))

        for alg in losses_dict[ts_name].keys():
            losses_dict[ts_name][alg] = np.cumsum(losses_dict[ts_name][alg]) / list(range(1, len(ts_test.values) + 1))
    
    return pred_dict, losses_dict

In [102]:
def plot_total_losses(losses_dict, ax, title, legend, number_to_plot=None):
    total_losses = {alg: np.zeros_like(losses) for alg, losses in losses_dict[next(iter(losses_dict))].items()}
            
    for ts_losses in losses_dict.values():
        for alg, losses in ts_losses.items():
            total_losses[alg] += losses
    
    for alg, losses in total_losses.items():
            total_losses[alg] = losses / len(losses_dict)
            
    total_losses_sorted = {k: v for k, v in sorted(total_losses.items(), key=lambda item: item[1].mean())}
            
    if not number_to_plot:
        number_to_plot = len(legend) - 2
    
    real_legend = []
    i = 0
    for alg in total_losses_sorted.keys():
        if i < number_to_plot:
            if alg == "AA":
                continue
            ax.plot(total_losses_sorted[alg])
            i += 1
            real_legend.append(legend[int(alg[-1]) - 1])
            print(f"{legend[int(alg[-1]) - 1]} mean loss: {total_losses_sorted[alg].mean()}")
        else:
            break
    
    ax.plot(total_losses_sorted['AA'])
    print(f"AA mean loss: {total_losses_sorted['AA'].mean()}")
    real_legend.append('AA')
        
    ax.set_title(title)
    ax.legend(real_legend, loc='lower right');